In [1]:
import random 
import numpy as np
import itertools
from functools import lru_cache

# Structure de données, affichage

In [2]:
# Représentation d'un pseudo-labyrinthe avec deux matrices de murs horizontaux et verticaux
class pseudoLabyrinthe:
    def __init__(self, n, m): # constructeur de la classe pseudolabyrinthe
        self.n = n
        self.m = m
        # Matrice de murs (True = mur, False = passage)
        self.murs_horizontaux = [[False for i in range(m)] for j in range(n-1)]
        self.murs_verticaux = [[False for i in range(n)] for j in range(m-1)]

    # Génère un pseudo-labyrinthe aléatoire
    def generer_aleatoire(self):
        for i in range(self.n - 1):
            for j in range(self.m):
                self.murs_horizontaux[i][j] = random.choice([True, False])
        for i in range(self.m - 1):
            for j in range(self.n):
                self.murs_verticaux[i][j] = random.choice([True, False])
    def enceinte(self):
        """
        affiche une cellule de taille M*N, avec la grille
        """
        k = line2d([(0,0), (self.m, 0), (self.m, self.n), (0, self.n), (0,0)], color="red", thickness=2, gridlines=True)
        return k
    def afficher(self):
        graphHorizontal = line2d([(0, 0), (0, 0)])
        graphVertical = line2d([(0, 0), (0, 0)])
        for i in range(len(self.murs_horizontaux)):
            for j in range(len(self.murs_horizontaux[i])):
                if (self.murs_horizontaux[i][j]):
                    graphHorizontal += line2d([(j, i+1), (j+1, i+1)], color="red", thickness=2) # j'ai échangé j et i
        for i in range(len(self.murs_verticaux)):
            for j in range(len(self.murs_verticaux[i])):
                if (self.murs_verticaux[i][j]):
                    graphVertical += line2d([(i+1, j), (i+1, j+1)], color="red", thickness=2)
        return graphHorizontal + graphVertical + self.enceinte()
    
    @lru_cache(maxsize=None)
    def estconnexe(self):
        """
        Vérifie si le labyrinthe est connexe
        """
        visited = [[False] * self.m for _ in range(self.n)]  # Initialisation de la matrice de visite
        stack = [(0, 0)]  # Pile pour le parcours DFS
        visited[0][0] = True  # Marquer la cellule de départ comme visitée
        
        # Parcours DFS
        while stack:
            i, j = stack.pop()
            # Parcourir les cellules voisines non visitées
            if i > 0 and not self.murs_horizontaux[i-1][j] and not visited[i-1][j]:
                stack.append((i-1, j))
                visited[i-1][j] = True
            if i < self.n - 1 and not self.murs_horizontaux[i][j] and not visited[i+1][j]:
                stack.append((i+1, j))
                visited[i+1][j] = True
            if j > 0 and not self.murs_verticaux[j-1][i] and not visited[i][j-1]:
                stack.append((i, j-1))
                visited[i][j-1] = True
            if j < self.m - 1 and not self.murs_verticaux[j][i] and not visited[i][j+1]:
                stack.append((i, j+1))
                visited[i][j+1] = True
        
        # Vérifier si toutes les cellules ont été visitées
        return all(all(row) for row in visited)

In [3]:
class presque_labyrinthe(pseudoLabyrinthe):
    def __init__(self, n, m):
        super().__init__(n, m)
        l = np.random.permutation(np.array([0 for i in range(n*m-1)] + [1 for i in range((2*n*m - n - m )-(n*m)+1)]))
        t = np.split(l, (n*(m-1),))
        self.murs_horizontaux = list(np.split(t[1], n-1))
        self.murs_verticaux = list(np.split(t[0], m-1))

In [4]:
"""
def generate_arrays(array):
    num_ones = array.count(1)
    num_zeros = array.count(0)
    size = len(array)

    # Générer tous les arrangements possibles des éléments
    arrangements = set(permutations(array, size))
    return filtered_arrays
""" 
#  Avant amélioration par ChatGPT

'\ndef generate_arrays(array):\n    num_ones = array.count(1)\n    num_zeros = array.count(0)\n    size = len(array)\n\n    # Générer tous les arrangements possibles des éléments\n    arrangements = set(permutations(array, size))\n    return filtered_arrays\n'

In [5]:
def generate_arrays_recursive(array, index, num_ones, num_zeros, current_array, result):
    # Cas de base : si tous les éléments ont été placés
    if index == len(array):
        result.append(current_array.copy())
        return

    # Si le prochain élément doit être 1
    if num_ones > 0:
        current_array[index] = 1
        generate_arrays_recursive(array, index + 1, num_ones - 1, num_zeros, current_array, result)

    # Si le prochain élément doit être 0
    if num_zeros > 0:
        current_array[index] = 0
        generate_arrays_recursive(array, index + 1, num_ones, num_zeros - 1, current_array, result)

def generate_arrays(array):
    num_ones = array.count(1)
    num_zeros = len(array) - num_ones
    result = []

    generate_arrays_recursive(array, 0, num_ones, num_zeros, [None] * len(array), result)

    return result

In [6]:
def toutlaby(n,m):
    count = 0
    k = generate_arrays([0 for i in range(n*m-1)] + [1 for i in range((2*n*m - n - m )-(n*m)+1)])
    lab = pseudoLabyrinthe(n,m)
    for perm in k:
        p = np.split(np.array(perm), (n*(m-1),))
        lab.murs_horizontaux = list(np.split(p[1], n-1))
        lab.murs_verticaux = list(np.split(p[0], m-1))
        if lab.estconnexe():
            count +=1
    return count

In [9]:
toutlaby(4,3)

0

Nombre de labyrinthe par taille:    

    2*2 = 4  
    2*3 = 3*2 = 15    
    2*4 = 4*2 = 256  
    3*3 = 192  
    3*4 = 4*3 = 2415  
    4*4 = 100352